# HW 期中 - 巴哈動畫瘋觀看數爬蟲

> Hsiao, Wei-pin @ NTNU Math

## 描述

[巴哈動畫瘋](https://ani.gamer.com.tw/)是台灣目前最大的動畫串流網站，其運機制有些類似中國的 bilibili 跟日本的 niconico (但不允許網友自行上傳動畫)，本期中專題試圖利用 Python 建立爬蟲，定期於動畫瘋官網抓取新番(指本季度正在放送的日本動畫節目)觀看類計次數。本專案分為下列幾個部分：

1. [`AGI_Daily_DWer.py`](https://github.com/PeterHsi/AniGamerInfo/blob/master/AGI_Daily_DWer.py) 爬蟲程式碼，將巴哈動畫瘋之觀看數字存檔為 `.csv` 檔。
   
   檔案命名規則為：`AGI_Daily_YYYYmmdd-HHMM.csv`，其中 `YYYY` 為西元年，`mm` 為月，`dd` 為日，`HH` 為時(24小時制)，`MM` 為分，時區為 UTC/GMT+8 (台北時區)。
   表格說明詳見： https://github.com/PeterHsi/AniGamerInfo/blob/master/README.md
   
2. 排程指令，以設定排程方式執行 `AGI_Daily_DWer.py` 。

目前成果放置於 [PeterHsi/AniGamerInfo](https://github.com/PeterHsi/AniGamerInfo) 此 GitHub 專案內(包含每小時蒐集一次之數據)，歡迎各位指教。
    
### 前人統計數字參考：
- [\[18秋\] 動畫瘋 秋番累積觀看次數總結算](https://www.ptt.cc/bbs/C_Chat/M.1546933824.A.B3A.html) (by PTTid: durantjosh)
- [Re: \[19冬\] 動畫瘋 冬番累積觀看次數走勢](https://www.ptt.cc/bbs/C_Chat/M.1554210297.A.6B5.html) (by PTTid: durantjosh)

---

## 1. 爬蟲程式碼

程式碼存取於 [`AGI_Daily_DWer.py`](https://github.com/PeterHsi/AniGamerInfo/blob/master/AGI_Daily_DWer.py)，以下為程式碼說明：

### 載入需要之套件

載入如下套件，最重要的為 BeautifulSoup (負責爬資料)與 pandas (負責製表)。

In [1]:
import os
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 取得巴哈動畫瘋官網 HTML 資料

In [2]:
reqAniGamer = requests.get('https://ani.gamer.com.tw/')
soup = BeautifulSoup(reqAniGamer.text, 'html.parser')
thisSeasonAnime = soup.find_all(attrs={"data-look": "本季新番"})

![](Data/HW_Mid_aniGamerHomepage.png)

直接檢查官網原始碼會發現，每個節目的資訊都掛在這邊，所以這邊是我們要集中處理之區域

In [2]:
numAni = [x for x in range(len(thisSeasonAnime))]
title = [str(thisSeasonAnime[i].find('p').string) for i in numAni]
vol = [thisSeasonAnime[i].find(
    class_="newanime-vol").string.replace("第 ", "").replace(" 集", "") for i in numAni]
update = [thisSeasonAnime[i].find(
    class_="newanime-date").string.replace(" 更新", "") for i in numAni]
unNumber = [thisSeasonAnime[i].find(
    class_="newanime-count") for i in numAni]
number = []
for str_temp in unNumber:
    posEnd = str(str_temp).find("</span>")
    posSta = str(str_temp).rfind(">", 0, posEnd)
    num_temp = str(str_temp)[posSta+1:posEnd].replace(",", "")
    number.append(num_temp)

### 將擷取資料彙整於表格

In [2]:
AnimeInfo = pd.DataFrame(
    {'title': title,
     'vol': vol,
     'update': update,
     'number': number
     })
AnimeInfo['date'] = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")

下載下來之資料粗略如下：

In [2]:
AnimeInfo.head()

,title,vol,update,number,date
0,戀愛幕末男友外傳-危機,2,04/12,2149,2019/04/12 18:28:46
1,Fight League 交鋒聯盟,9,04/11,57250,2019/04/12 18:28:46
2,一拳超人 (第二季),13,04/10,356899,2019/04/12 18:28:46
3,強襲魔女 501 部隊出動！,1,04/10,14948,2019/04/12 18:28:46
4,鑽石王牌 act2,2,04/09,58901,2019/04/12 18:28:46


### 存檔

In [3]:
tsp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
if not os.path.exists(pathlib.PurePath('DailyData/'))
    os.makedirs(pathlib.PurePath('DailyData/'))
AnimeInfo.to_csv(pathlib.PurePath('DailyData/AniGamerInfo '+tsp+'.csv'))

不使用 `PurePath` 的版本

In [ ]:
# tsp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
# if not os.path.exists('DailyData/'):
#     os.makedirs('DailyData/')
# AnimeInfo.to_csv('DailyData/AGI_Daily_'+tsp+'.csv')

## 2. 排程指令

設定完對應的 repo 之後，利用電腦之排程功能執行下列命令

In [ ]:
# cd /your path here/
# python3 AGI_Daily_DWer.py
# git commit -m "Schedule data update."
# git push -u origin master

### 附錄：Windows 的排程功能

首先請找到「工作排程器(Task Scheduler)」這個內建軟體

![](Data/HW_Mid_winSche01.png)

然後按「建立工作」

![](Data/HW_Mid_winSche02.png)

然後新增動作(將排程指以純文字檔存檔並給予 `.bat` 副檔名並以下圖方式選定執行)

![](Data/HW_Mid_winSche03.png)

設定觸發條件

![](Data/HW_Mid_winSche04.png)

最後設定適當權限並儲存排程

![](Data/HW_Mid_winSche05.png)

之後檢查是否有依時工作再調整就好了!